In [2]:
from google import genai
import os
import time

In [3]:
client = genai.Client(api_key=os.getenv("GEMINI_KEY"))

In [5]:
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain how AI works in a few words"
)

In [6]:
print(response.text)

AI uses algorithms to learn from data and make predictions or decisions.



In [14]:
# First test the code with a single json file

with open("All_Conversations/c67274cc-d920-467a-80cc-476c9dd280d4.json", "r") as f:
    conversation = f.read()

    prompt = """
    You are a helpful assistant. Your task is to identify any profane words in the conversation below.
    If you find any, please list them. If there are no profane words, respond with a blank list.

    Give output strictly in the form of a python list:
    ["word1", "word2", ...]
    or [] if there are no profane words.

    Only return the list and nothing else.

    Here is the conversation:
    """ + conversation

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
    )

    print(response.text)

    time.sleep(3)

['hell', 'shit', 'ass', 'dumbass']


In [12]:
# read json files from All_Conversations
# make api calls sending json content and asking it to detect all profane words from the conversation
# pause for 3 seconds between each call to avoid rate limit

# each json file has content as follows
# Each json file is a conversation with multiple messages structured as follows
# [
#     {
#         "speaker": "Agent",
#         "text": "Hello, is this Mr. Johnson? This is Lisa calling from XYZ Collections. How are you today?",
#         "stime": 0,
#         "etime": 7
#     },
#     {
#         "speaker": "Customer",
#         "text": "I'm sorry, but I think you have the wrong person. My name is Sarah.",
#         "stime": 6.5,
#         "etime": 12
#     },
#     {
#         "speaker": "Agent",
#         "text": "Oh, I apologize for the confusion, Sarah. I'm reaching out about a debt related to an outstanding balance with Definite Bank.",
#         "stime": 11,
#         "etime": 19
#     },
#     {
#         "speaker": "Customer",
#         "text": "I don't have any account with Definite Bank. You might want to check your records.",
#         "stime": 18,
#         "etime": 24
#     },
#     {
#         "speaker": "Agent",
#         "text": "Thank you for letting me know. I will make a note to update our records.",
#         "stime": 23,
#         "etime": 30
#     },
#     {
#         "speaker": "Customer",
#         "text": "I appreciate that. Is there anything else I need to do?",
#         "stime": 29,
#         "etime": 34
#     },
#     {
#         "speaker": "Agent",
#         "text": "No, that's all. I'm sorry for any inconvenience caused. Have a great day!",
#         "stime": 33,
#         "etime": 40
#     },
#     {
#         "speaker": "Customer",
#         "text": "Thank you, you too!",
#         "stime": 39,
#         "etime": 42
#     }
# ]

# make profane_words a set
profane_words = set()

for file in os.listdir("All_Conversations"):
    if file.endswith(".json"):
        with open(os.path.join("All_Conversations", file), "r") as f:
            conversation = f.read()

            prompt = """
            You are a helpful assistant. Your task is to identify any profane words in the conversation below.
            If you find any, please list them. If there are no profane words, respond with a blank list.

            Give output strictly in the form of a python list:
            ["word1", "word2", ...]
            or [] if there are no profane words.

            Only return the list and nothing else.

            Here is the conversation:
            """ + conversation

            response = client.models.generate_content(
                model="gemini-2.0-flash",
                contents=prompt,
            )

            # trust that the response is a valid list and push the response to profane_words
            try:
                words = eval(response.text)
                if isinstance(words, list):
                    # add the words to the set
                    for word in words:
                        if isinstance(word, str):
                            profane_words.add(word.strip())
                else:
                    print(f"Unexpected response format for file {file}: {response.text}")
            except Exception as e:
                print(f"Error processing file {file}: {e}")

            time.sleep(3) # to prevent rate limit issues

Error processing file c67274cc-d920-467a-80cc-476c9dd280d4.json: invalid syntax (<string>, line 1)


In [13]:
profane_words = list(profane_words)
print("Profane words found in the conversations:")
print(profane_words)

Profane words found in the conversations:
['f***ing', 'crap', 'BS', 'fuck', 'dead', 'joke', 'f******', 'F***', 'idiots', 'goddamn', 'Screw', 'f**k', 'screwed', 'ass', 'jerking', 'a**', 'sick', 'damn', 'shit', 'idiot', 'bullshit', 'hell', 'pissing', 'shove', 'stupid', 'assholes', 'screw', 'bullsh*t', 'sh*t', 's***', 'freaking', 's**t', 'shitload', 'f***']


In [15]:
profane_words = set([
    'fucking', 'crap', 'BS', 'fuck', 'dead', 'joke',
    'idiots', 'goddamn', 'screw', 'fuck', 'screwed', 'ass', 'jerking',
    'sick', 'damn', 'shit', 'idiot', 'bullshit', 'hell', 'pissing',
    'shove', 'stupid', 'assholes', 'screw', 'bullshit',
    'freaking', 'shitload', 'dumbass'
])

In [16]:
list(profane_words)

['BS',
 'crap',
 'fuck',
 'dead',
 'joke',
 'idiots',
 'goddamn',
 'screwed',
 'ass',
 'jerking',
 'sick',
 'damn',
 'shit',
 'idiot',
 'bullshit',
 'hell',
 'pissing',
 'shove',
 'stupid',
 'assholes',
 'fucking',
 'screw',
 'freaking',
 'shitload',
 'dumbass']

The profane words list has been extracted using LLM to be used with regex match as well.